## 🌐 Connect Colab to Google Drive

In [1]:
from google.colab import drive

drive.mount("/gdrive")
%cd /gdrive/My Drive
%cd [2024-2025] AN2DL Homework 2

Mounted at /gdrive
/gdrive/My Drive
/gdrive/My Drive/[2024-2025] AN2DL Homework 2


## ⚙️ Import Libraries

In [2]:
from datetime import datetime

import numpy as np
import pandas as pd
import logging
import random

import tensorflow as tf
from tensorflow import keras as tfk
tfk.config.enable_unsafe_deserialization()
from tensorflow.keras import layers as tfkl
from tensorflow.keras.layers import Layer
from sklearn.model_selection import train_test_split
from scipy.stats import mode

import matplotlib.pyplot as plt
%matplotlib inline

seed = 29
np.random.seed(seed)
tf.random.set_seed(seed)

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

## ⏳ Load the Datasets

In [3]:
data = np.load("mars_for_students.npz")

training_set = data["training_set"]
X_train = training_set[:, 0]
y_train = training_set[:, 1]

X_test = data["test_set"]

print(f"Training X shape: {X_train.shape}")
print(f"Training y shape: {y_train.shape}")
print(f"Test X shape: {X_test.shape}")

# Add color channel and rescale pixels between 0 and 1
X_train = X_train[..., np.newaxis] / 255.0
X_test = X_test[..., np.newaxis] / 255.0

input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))

print(f"Input shape: {input_shape}")
print(f"Number of classes: {num_classes}")

Training X shape: (2615, 64, 128)
Training y shape: (2615, 64, 128)
Test X shape: (10022, 64, 128)
Input shape: (64, 128, 1)
Number of classes: 5


## 🔍 Inspect the training dataset

In [ ]:
# Calculate prevalent labels
y_train_labels = mode(y_train, axis=(1, 2))[0].flatten()

print(f"Shape X_train: {X_train.shape}")
print(f"Shape y_train_labels: {y_train_labels.shape}")

# List all unique labels to check correctness
unique_labels = np.unique(y_train)
print(f"Unique classes: {unique_labels}")

# Plot images in batches
def plot_images(X, y, start_index=0, images_per_row=10, images_per_col=10):
    fig, axes = plt.subplots(images_per_col, images_per_row, figsize=(15, 15))
    for i in range(images_per_row * images_per_col):
        idx = start_index + i
        if idx >= len(X):
            break
        ax = axes[i // images_per_row, i % images_per_row]
        ax.imshow(X[idx], cmap="gray")
        ax.set_title(f"Class: {y[idx]}")
        ax.axis("off")
    plt.tight_layout()
    plt.show()

# Plot a sample image from each class
def plot_one_sample_per_class(X, y, y_mask, classes):
    for label in classes:
        for i in range(len(y_mask)):
            if label in np.unique(y_mask[i]):
                plt.figure()
                plt.imshow(X[i], cmap="gray")
                plt.title(f"Class: {label}")
                plt.axis("off")
                plt.show()
                break

plot_one_sample_per_class(X_train, y_train_labels, y_train, unique_labels)

# Plot all images
images_per_row = 10
images_per_col = 10
images_per_page = images_per_row * images_per_col
num_images = X_train.shape[0]

for start_idx in range(0, num_images, images_per_page):
    plot_images(X_train, y_train_labels, start_index=start_idx, images_per_row=images_per_row, images_per_col=images_per_col)

## ❌ Remove outliers from dataset

In [ ]:
# Lists to contain filtered elements
X_train_filtered = []
y_train_filtered = []

for i in range(len(y_train)):
    label = y_train[i].argmax() if y_train.ndim > 1 else y_train[i]
    if label != 415:
        # Add to filtered dataset the non-alien images
        X_train_filtered.append(X_train[i])
        y_train_filtered.append(y_train[i])

# Convert lists to numpy arrays
X_train_filtered = np.array(X_train_filtered)
y_train_filtered = np.array(y_train_filtered)

print(f"Shape X_train_filtered: {X_train_filtered.shape}")
print(f"Shape y_train_filtered: {y_train_filtered.shape}")
print(f"Unique classes: {np.unique(y_train_filtered)}")

Shape X_train_filtered: (2505, 64, 128, 1)
Shape y_train_filtered: (2505, 64, 128)
Unique classes: [0. 1. 2. 3. 4.]


## 🔍 Inspect the filtered training dataset

In [ ]:
num_images_filtered = X_train_filtered.shape[0]
y_train_filtered_labels = mode(y_train_filtered, axis=(1, 2))[0].flatten()

# Plot the filtered dataset
for start_idx in range(0, num_images_filtered, images_per_page):
    plot_images(X_train_filtered, y_train_filtered_labels, start_index=start_idx, images_per_row=images_per_row, images_per_col=images_per_col)

## 🧮 Define network parameters

In [ ]:
# Set batch size for training
batch_size = 16

# Set learning rate for the optimizer
learning_rate = 1e-3

# Set early stopping patience threshold
patience = 10

# Set maximum number of training epochs
epochs = 100

## ✂ Split into Training and Validation Sets

In [ ]:
# Split the training dataset to get a validation set
X_train, X_val, y_train, y_val = train_test_split(
    X_train_filtered,
    y_train_filtered,
    test_size=0.2,
    random_state=seed)

# Print the shapes of the resulting sets
print('Training set shape:\t', X_train.shape, y_train.shape)
print('Validation set shape:\t', X_val.shape, y_val.shape)

Training set shape:	 (2004, 64, 128, 1) (2004, 64, 128)
Validation set shape:	 (501, 64, 128, 1) (501, 64, 128)


## 🔄 Preprocess Dataset

In [ ]:
def augment_data(image, label):
    # Geometric Transformations
    image = tf.image.random_flip_left_right(image)
    label = tf.image.random_flip_left_right(label)

    image = tf.image.random_flip_up_down(image)
    label = tf.image.random_flip_up_down(label)

    # Chromatic Transformations
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)

    return image, label

def preprocess_image(image):
    image = tf.expand_dims(image, axis=-1) if len(image.shape) == 2 else image
    image = tf.cast(image, tf.float32)
    return image

def preprocess_label(label):
    label = tf.expand_dims(label, axis=-1) if len(label.shape) == 2 else label
    label = tf.cast(label, tf.int32)
    return label

def preprocess_data(image, label):
    image = preprocess_image(image)
    label = preprocess_label(label)
    return image, label

In [ ]:
# Original dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.map(preprocess_data, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Augmented dataset
augmented_dataset = train_dataset.map(lambda x, y: augment_data(x, y), num_parallel_calls=tf.data.AUTOTUNE)

# Combined dataset, having both augmented and original dataset
combined_dataset = train_dataset.concatenate(augmented_dataset)
combined_dataset = combined_dataset.shuffle(buffer_size=len(X_train))
combined_dataset = combined_dataset.prefetch(tf.data.AUTOTUNE)

# Validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.map(preprocess_data, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

## 🔍 Plot Datasets

In [ ]:
def plot_from_dataset(dataset, title, num_images=10):
    # Imposta il layout orizzontale
    fig, axes = plt.subplots(1, num_images, figsize=(num_images * 2, 3))
    fig.suptitle(title, fontsize=16, y=1.05)
    count = 0

    for batch in dataset:
        images, label_maps = batch  # Estrai immagini e feature map
        for i in range(len(images)):
            if count >= num_images:
                break

            # Prepara l'immagine
            image = images[i].numpy()
            if image.shape[-1] == 1:  # Scala di grigi
                image = tf.squeeze(image, axis=-1).numpy()

            # Mostra l'immagine
            axes[count].imshow(image, cmap='gray' if image.ndim == 2 else None, aspect='auto')
            axes[count].axis('off')

            count += 1

        if count >= num_images:
            break

    plt.tight_layout()
    plt.show()

plot_from_dataset(train_dataset, "Train Dataset without Augmentation", num_images=10)
plot_from_dataset(augmented_dataset, "Train Dataset with Augmentation", num_images=10)
plot_from_dataset(combined_dataset, "Combined Dataset", num_images=10)

## 🔨 Build the model

In [ ]:
class ResizeLayer(Layer):
    def __init__(self, target_size, **kwargs):
        super(ResizeLayer, self).__init__(**kwargs)
        self.target_size = target_size

    def call(self, inputs):
        return tf.image.resize(inputs, self.target_size)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.target_size[0], self.target_size[1], input_shape[-1])

class AdaptShapeLayer(tfkl.Layer):
    def __init__(self, **kwargs):
        super(AdaptShapeLayer, self).__init__(**kwargs)

    def call(self, tensor, target_tensor):
        target_shape = tf.shape(target_tensor)[1:3]
        tensor = tf.image.resize(tensor, target_shape, method='bilinear')
        return tensor

def adapt_channels(tensor, target_channels):
    conv = tfkl.Conv2D(target_channels, kernel_size=1, padding='same')
    return conv(tensor)

In [ ]:
def unet_block(input_tensor, filters, kernel_size=3, stack=4, name=''):
    x = input_tensor
    adapt_shape = AdaptShapeLayer()
    residual = adapt_channels(input_tensor, filters)

    for i in range(stack):
        x = tfkl.Conv2D(filters, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal')(x)
        x = tfkl.BatchNormalization()(x)
        x = se_block(x)
        x = tfkl.Activation(tf.nn.leaky_relu)(x)
        x = tfkl.Conv2D(filters, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal')(x)
        x = tfkl.BatchNormalization()(x)
        x = se_block(x)
        x = tfkl.Activation(tf.nn.leaky_relu)(x)
        x = tfkl.SpatialDropout2D(0.2)(x)

    residual = adapt_shape(residual, x)
    x = tfkl.Add()([x, residual])
    return x

def dense_block(input_tensor, filters, kernel_size=3, growth_rate=32, num_layers=4):
    x = input_tensor
    for i in range(num_layers):
        conv = tfkl.Conv2D(growth_rate, kernel_size, padding='same', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
        conv = tfkl.BatchNormalization()(conv)
        conv = tfkl.Activation(tf.nn.leaky_relu)(conv)
        x = tfkl.Concatenate()([x, conv])
    return x

def se_block(input_tensor, reduction_ratio=16):
    filters = input_tensor.shape[-1]
    se = tfkl.GlobalAveragePooling2D()(input_tensor)
    se = tfkl.Dense(filters // reduction_ratio, activation='relu', kernel_initializer='he_normal')(se)
    se = tfkl.Dense(filters, activation='sigmoid', kernel_initializer='he_normal')(se)
    se = tfkl.Reshape((1, 1, filters))(se)
    return tfkl.Multiply()([input_tensor, se])

def refinement_block(x, filters):
    for _ in range(2):
        x = tfkl.Conv2D(filters, (3, 3), padding="same", activation="relu")(x)
        x = tfkl.BatchNormalization()(x)
    return x

# Parallel Dilated Convolutions
def par_dil_conv(input_tensor, filters, kernel_size=3, dilation_rates=(1, 2, 4)):
    branches = []
    for rate in dilation_rates:
        branch = tfkl.Conv2D(filters, kernel_size=kernel_size, dilation_rate=rate, padding='same', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(input_tensor)
        branch = tfkl.BatchNormalization()(branch)
        branch = tfkl.Activation(tf.nn.leaky_relu)(branch)
        branches.append(branch)

    output = tfkl.Concatenate()(branches)
    return output

# Atrous Spatial Pyramid Pooling
def aspp(input_tensor, filters):
    branch1 = tfkl.Conv2D(filters, kernel_size=1, padding='same', activation='relu')(input_tensor)
    branch2 = tfkl.Conv2D(filters, kernel_size=3, dilation_rate=6, padding='same', activation='relu')(input_tensor)
    branch3 = tfkl.Conv2D(filters, kernel_size=3, dilation_rate=12, padding='same', activation='relu')(input_tensor)
    branch4 = tfkl.Conv2D(filters, kernel_size=3, dilation_rate=18, padding='same', activation='relu')(input_tensor)

    # Pooling branch
    pooling = tfkl.GlobalAveragePooling2D()(input_tensor)
    pooling = tfkl.Reshape((1, 1, input_tensor.shape[-1]))(pooling)
    pooling = tfkl.Conv2D(filters, kernel_size=1, padding='same', activation='relu')(pooling)
    pooling = tfkl.UpSampling2D(size=(input_tensor.shape[1], input_tensor.shape[2]))(pooling)

    output = tfkl.Concatenate()([branch1, branch2, branch3, branch4, pooling])
    return tfkl.Conv2D(filters, kernel_size=1, padding='same', activation='relu')(output)


def bottleneck_layer(input_tensor, filters, reduction_ratio=4, dilation_rates=(1, 2, 4)):
    adapt_shape = AdaptShapeLayer()

    residual = adapt_channels(input_tensor, filters)

    # Bottleneck Structure
    reduced_filters = filters // reduction_ratio
    x = tfkl.Conv2D(reduced_filters, kernel_size=3, padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(input_tensor)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.SeparableConv2D(filters, kernel_size=3, padding='same', activation='relu')(x)
    x = tfkl.BatchNormalization()(x)
    x = par_dil_conv(x, filters=filters, kernel_size=3, dilation_rates=dilation_rates)
    x = tfkl.Conv2D(filters, kernel_size=1, padding='same', kernel_initializer='he_normal')(x)
    x = tfkl.BatchNormalization()(x)
    x = se_block(x)
    x = tfkl.Dropout(0.3)(x)
    residual = adapt_shape(residual, x)
    x = tfkl.Add()([x, residual])
    return x

def refinement_block(x, filters):
    x = tfkl.Conv2D(filters, (3, 3), padding="same", activation="relu")(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Conv2D(filters, (3, 3), padding="same", activation="relu")(x)
    return x

## 🔨 Build UNets

In [ ]:
def create_global_unet(input_shape, num_classes, dilation_rates):
    inputs = tfkl.Input(shape=input_shape)

    # Encoder
    x1 = unet_block(inputs, filters=8, name='global_enc1')
    p1 = tfkl.MaxPooling2D((2, 2))(x1)
    x2 = unet_block(p1, filters=16, name='global_enc2')
    p2 = tfkl.MaxPooling2D((2, 2))(x2)
    x3 = unet_block(p2, filters=32, name='global_enc3')
    p3 = tfkl.MaxPooling2D((2, 2))(x3)
    x4 = unet_block(p3, filters=64, name='global_enc4')
    p4 = tfkl.MaxPooling2D((2, 2))(x4)

    # Bottleneck
    b = bottleneck_layer(p4, filters=128, dilation_rates=dilation_rates)
    b = aspp(b, filters=256)  # ASPP

    # Decoder
    u4 = tfkl.UpSampling2D((2, 2))(b)
    c4 = tfkl.Concatenate()([u4, x4])
    d4 = refinement_block(c4, filters=64)
    u3 = tfkl.UpSampling2D((2, 2))(d4)
    c3 = tfkl.Concatenate()([u3, x3])
    d3 = refinement_block(c3, filters=32)
    u2 = tfkl.UpSampling2D((2, 2))(d3)
    c2 = tfkl.Concatenate()([u2, x2])
    d2 = refinement_block(c2, filters=16)
    u1 = tfkl.UpSampling2D((2, 2))(d2)
    c1 = tfkl.Concatenate()([u1, x1])
    d1 = refinement_block(c1, filters=8)

    outputs = tfkl.Conv2D(num_classes, kernel_size=1, activation='softmax')(d1)

    model = tfk.Model(inputs=inputs, outputs=outputs, name="Global_U-Net")
    return model

def create_local_unet(input_shape, num_classes, dilation_rates):
    inputs = tfkl.Input(shape=input_shape)

    # Encoder
    x1 = unet_block(inputs, filters=8, name='local_enc1')
    p1 = tfkl.MaxPooling2D((2, 2))(x1)
    x2 = unet_block(p1, filters=16, name='local_enc2')
    p2 = tfkl.MaxPooling2D((2, 2))(x2)
    x3 = unet_block(p2, filters=32, name='local_enc3')
    p3 = tfkl.MaxPooling2D((2, 2))(x3)
    x4 = unet_block(p3, filters=64, name='local_enc4')
    p4 = tfkl.MaxPooling2D((2, 2))(x4)

    # Bottleneck
    b = bottleneck_layer(p4, filters=128, dilation_rates=dilation_rates)
    b = par_dil_conv(b, filters=256, kernel_size=3, dilation_rates=(1,2,4))

    # Decoder
    u4 = tfkl.UpSampling2D((2, 2))(b)
    c4 = tfkl.Concatenate()([u4, x4])
    d4 = refinement_block(c4, filters=64)
    u3 = tfkl.UpSampling2D((2, 2))(d4)
    c3 = tfkl.Concatenate()([u3, x3])
    d3 = refinement_block(c3, filters=32)
    u2 = tfkl.UpSampling2D((2, 2))(d3)
    c2 = tfkl.Concatenate()([u2, x2])
    d2 = refinement_block(c2, filters=16)
    u1 = tfkl.UpSampling2D((2, 2))(d2)
    c1 = tfkl.Concatenate()([u1, x1])
    d1 = refinement_block(c1, filters=8)

    outputs = tfkl.Conv2D(num_classes, kernel_size=1, activation='softmax')(d1)

    model = tfk.Model(inputs=inputs, outputs=outputs, name="Local_U-Net")
    return model

In [ ]:
# Dice Loss
def dice_loss(y_true, y_pred, smooth=1e-6):
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=num_classes, axis=-1)
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)
    return 1 - dice

# Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    gamma = tf.constant(gamma, dtype=tf.float32)
    alpha = tf.constant(alpha, dtype=tf.float32)

    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        y_true_one_hot = tf.one_hot(tf.cast(y_true, tf.int32), depth=y_pred.shape[-1])
        y_true_one_hot = tf.squeeze(y_true_one_hot, axis=-2) if len(y_true_one_hot.shape) > len(y_pred.shape) else y_true_one_hot
        cross_entropy = -y_true_one_hot * tf.keras.backend.log(y_pred)
        weight = alpha * tf.math.pow((1 - y_pred), gamma)
        loss = weight * cross_entropy
        return tf.reduce_mean(tf.reduce_sum(loss, axis=-1))

    return focal_loss_fixed

# Combined Loss
def combined_loss(y_true, y_pred, dice_weight=0.5, focal_weight=0.5):
    dice = dice_loss(y_true, y_pred, smooth=1e-6)
    focal = focal_loss(gamma=2.0, alpha=0.25)(y_true, y_pred)
    combined = dice_weight * dice + focal_weight * focal

    return combined

In [ ]:
inputs = tfkl.Input(shape=input_shape)

# Crea le due reti separatamente
global_unet = create_global_unet(input_shape, num_classes, (2, 4, 8))
local_unet = create_local_unet(input_shape, num_classes, (1, 2, 4))

# Input condiviso
inputs = tfkl.Input(shape=input_shape)

# Passaggio dell'input alle due reti
global_output = global_unet(inputs)
local_output = local_unet(inputs)

# Fusione dei risultati (concatenazione)
merged = tfkl.Concatenate()([global_output, local_output])

# Un piccolo blocco di affinamento finale
fused = refinement_block(merged, filters=64)

# Output finale
final_output = tfkl.Conv2D(num_classes, kernel_size=1, activation='softmax')(fused)

model = tfk.Model(inputs=inputs, outputs=final_output)

# Define the MeanIoU ignoring the background class
mean_iou = tfk.metrics.MeanIoU(num_classes=num_classes, ignore_class=0, sparse_y_pred=False, name='mean_iou')

# Compile the model
optimizer = tfk.optimizers.AdamW(learning_rate=learning_rate, weight_decay=1e-5)

loss = combined_loss

# Compile the model
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=[mean_iou]
)

model.summary()

Model: "Double_U-Net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, 64, 128, 1)     │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Global_U-Net (Functional) │ (None, 64, 128, 5)     │     74,568,741 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Local_U-Net (Functional)  │ (None, 64, 128, 5)     │     76,078,373 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_13            │ (None, 64, 128, 10)    │              0 │ Global_U-Net[0][0],    │
│ (Concatenate)             │                        │                │ Local_U-Net[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_152 (Conv2D)       │ (None, 64, 128, 64)    │          5,824 │ concatenate_13[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_124   │ (None, 64, 128, 64)    │            256 │ conv2d_152[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_153 (Conv2D)       │ (None, 64, 128, 64)    │         36,928 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_154 (Conv2D)       │ (None, 64, 128, 5)     │            325 │ conv2d_153[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 150,690,447 (574.84 MB)

 Trainable params: 150,632,719 (574.62 MB)

 Non-trainable params: 57,728 (225.50 KB)

## 🛠️ Train and Save the Model

In [ ]:
# Create an EarlyStopping callback
early_stopping = tfk.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=patience,
    restore_best_weights=True
)

# Create a LearningRate Scheduler
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_mean_iou', factor=0.5, patience=5, min_lr=1e-5
)

# Store the callbacks in a list
callbacks = [early_stopping, lr_scheduler]

In [ ]:
history = model.fit(
    combined_dataset,
    epochs=epochs,
    validation_data=val_dataset,
    batch_size=batch_size,
    callbacks=callbacks
).history

Epoch 1/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 492s 888ms/step - loss: 0.8645 - mean_iou: 0.1116 - val_loss: 0.4745 - val_mean_iou: 0.0939 - learning_rate: 0.0010
Epoch 2/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 35s 137ms/step - loss: 0.4703 - mean_iou: 0.1202 - val_loss: 0.4427 - val_mean_iou: 0.1046 - learning_rate: 0.0010
Epoch 3/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 33s 132ms/step - loss: 0.4426 - mean_iou: 0.1352 - val_loss: 0.4782 - val_mean_iou: 0.0768 - learning_rate: 0.0010
Epoch 4/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 33s 131ms/step - loss: 0.4207 - mean_iou: 0.1703 - val_loss: 0.4188 - val_mean_iou: 0.1762 - learning_rate: 0.0010
Epoch 5/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 33s 133ms/step - loss: 0.4062 - mean_iou: 0.2066 - val_loss: 0.3977 - val_mean_iou: 0.2392 - learning_rate: 0.0010
Epoch 6/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 35s 137ms/step - loss: 0.3778 - mean_iou: 0.2523 - val_loss: 0.3622 - val_mean_iou: 0.2653 - learning_rate: 0.0010
Epoch 7/100
251/251 ━━━━━━━━━━━━━━━━━━━━ 34s 136ms/step - loss:

In [ ]:
timestep_str = datetime.now().strftime("%y%m%d_%H%M%S")
model_filename = f"model_{timestep_str}.keras"
model.save(model_filename)
del model

## 📊 Test the model



In [ ]:
model = tfk.models.load_model(model_filename, custom_objects={
        "ResizeLayer": ResizeLayer,
        'dice_loss': dice_loss,
        'focal_loss': focal_loss,
        'combined_loss': combined_loss,
        'unet_block': unet_block,
        'dense_block': dense_block,
        'par_dil_conv': par_dil_conv,
        'bottleneck_layer': bottleneck_layer,
        'se_block': se_block
    }
)

preds = model.predict(X_test)
preds = np.argmax(preds, axis=-1)
print(f"Predictions shape: {preds.shape}")

314/314 ━━━━━━━━━━━━━━━━━━━━ 30s 57ms/step
Predictions shape: (10022, 64, 128)


## 💾 Save the predictions

In [ ]:
def y_to_df(y) -> pd.DataFrame:
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

In [ ]:
# Create the csv submission file
timestep_str = model_filename.replace("model_", "").replace(".keras", "")
submission_filename = f"submission_{timestep_str}.csv"
submission_df = y_to_df(preds)
submission_df.to_csv(submission_filename, index=False)